# ReverseNumber Example - two users

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import os, json

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread, run_get_instance_worker_thread

def status(cl):
    colink_utils.print_served_flows(cl, print_values=False)
    print("\nAll flow instances:")
    colink_utils.print_flow_instances(cl)

/Users/nicolasbaldwin/opt/miniconda3/envs/coflows/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Create two users

In [4]:
cls = colink_utils.start_colink_server_with_users(num_users=2)

# USER 0

In [5]:
run_dispatch_worker_thread(cls[0])
run_get_instance_worker_thread(cls[0])

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
get_instances worker started in attached thread for user  03b1fa8528934896b6e2fa9ad609cd83a0efa419c327673c4b5bbfeb56041c2ae7


In [6]:
serve_utils.serve_flow(
    cl=cls[0],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic",
    singleton=True,
)

Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: True



True

# USER 1

In [7]:
run_dispatch_worker_thread(cls[1])
run_get_instance_worker_thread(cls[1])

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
get_instances worker started in attached thread for user  03eeb068191828bdfd452f7cbda672f92a3639c9777b2757d73a8fc8b8b230b2c6


In [8]:
serve_utils.serve_flow(
    cl=cls[1],
    flow_class_name="ReverseNumberFlowModule.ReverseNumbeAtomicFlow",
    flow_endpoint="reverse_number_atomice",
)

Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False



True

In [9]:
# since we are creating new dummy users on every run, we inject user_id into config manually
# typically this would be hardcoded in yaml config
cfg_overrides = {
    "subflows_config": {
        "first_reverse_flow": {
            "user_id": cls[0].get_user_id()
        },
        "second_reverse_flow": {
            "user_id": cls[0].get_user_id()
        }
    }
}

flow = serve_utils.get_flow_instance(
    cl=cls[1],
    flow_endpoint="reverse_number_sequential",
    user_id="local",
    config_overrides = cfg_overrides,
)


~~~ serving get_instances request ~~~
task_id = 5619caf6-ab74-4dae-8ebf-7b1f8fd86e99
get_instance_calls: [['first_reverse_flow', 'reverse_number_atomic', {'flow_class_name': 'ReverseNumberFlowModule.ReverseNumberAtomicFlow', 'singleton': True, 'name': 'ReverseNumberFirst', 'description': 'A flow that takes in a number and reverses it.'}], ['second_reverse_flow', 'reverse_number_atomic', {'flow_class_name': 'ReverseNumberFlowModule.ReverseNumberAtomicFlow', 'name': 'ReverseNumberSecond', 'description': 'A flow that takes in a number and reverses it AGAIN.'}]]
Mounted ea0db682-13c8-4902-8972-199b035fb7dd at flows:reverse_number_atomic:mounts:03eeb068191828bdfd452f7cbda672f92a3639c9777b2757d73a8fc8b8b230b2c6:ea0db682-13c8-4902-8972-199b035fb7dd
Fetched singleton ea0db682-13c8-4902-8972-199b035fb7dd

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_atomic
flow_id: ea0db682-13c8-4902-8972-199b035fb7dd
owner_id: 03eeb068191828bdfd452f7cbda672f92a3639c9777b2757d73a8fc8b8b230b2c6
message_p


~~~ get_instances initiator ~~~
task_id = 5619caf6-ab74-4dae-8ebf-7b1f8fd86e99
Received subflow instances: {
    "first_reverse_flow": {
        "flow_id": "ea0db682-13c8-4902-8972-199b035fb7dd",
        "successful": true,
        "message": "Fetched local flow instance."
    },
    "second_reverse_flow": {
        "flow_id": "ea0db682-13c8-4902-8972-199b035fb7dd",
        "successful": true,
        "message": "Fetched local flow instance."
    }
}

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_sequential
flow_id: 729cc81e-8e0d-461e-9b0f-1edbdae8988d
owner_id: local
message_paths: ['push_tasks:4d6390ac-c294-4915-b5ec-68fe2f795f41:msg']
parallel_dispatch: False

Input message source: Proxy_reverse_number_sequential
Called ReverseNumberSequential 
 state {'current_call': 'first_reverse_flow'}

~~~ Dispatch task ~~~
flow_endpoint: reverse_number_sequential
flow_id: 729cc81e-8e0d-461e-9b0f-1edbdae8988d
owner_id: local
message_paths: ['push_tasks:62b2d3bc-5399-4759-8ac2-f4e530a2ecb

Mounted 729cc81e-8e0d-461e-9b0f-1edbdae8988d at flows:reverse_number_sequential:mounts:local:729cc81e-8e0d-461e-9b0f-1edbdae8988d


In [10]:
input_data = {"id": 0, "number": 1234}
    
input_message = flow.package_input_message(input_data)
future = flow.get_reply_future(input_message)
reply_data = future.get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

Data sent:
 {'id': 0, 'number': 1234} 

REPLY:
 {'output_number': 1234} 



# Observe storage of both users

User 0 has a serve endpoint for ReverseNumberAtomic flow and a singleton instance of that flow.
User 1 has a serve endpoint for ReverseNumberSequential flow and an instance of that flow whose subflows are remote instances of the ReverseNumberAtomic at user 0.

In [10]:
status(cls[0])

 reverse_number_atomic/
   flow_class_name
   singleton
   parallel_dispatch
   mounts/
     02d29bc3e276de86016c350d3e3b637148eda4af310c0f84405c0f138365aab669/
       c9257714-8465-4951-8c15-e4aefaa58cc3/
         config_overrides
         state
         init
   init
   default_dispatch_point

All flow instances:
 c9257714-8465-4951-8c15-e4aefaa58cc3


In [11]:
status(cls[1])

 reverse_number_sequential/
   default_dispatch_point
   init
   flow_class_name
   parallel_dispatch
   singleton
   mounts/
     local/
       3180a6d9-7eb5-4ec3-8c7b-3c9c08f4fbdc/
         state
         init
         config_overrides

All flow instances:
 3180a6d9-7eb5-4ec3-8c7b-3c9c08f4fbdc
